## **Imports and Setup**

In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, LambdaCallback
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import json
#from copy import copy
from tensorflow.keras.applications import VGG16 

print(tf.__version__)

2.4.0


## **Mount Notebook on Google Drive**
Mount and give read/write access directly to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Read & Pre-Process Data**

Read Data

In [3]:
data_folder = '/content/drive/My Drive/Colab Notebooks/Cloud Classification/data/swimcat/'

def load_images_from_folder(folder):
  images = []
  for filename in os.listdir(folder):
    img = cv2.imread(os.path.join(folder,filename))
    if img is not None:
      images.append(img.astype('uint8')) # Store in 'uint8' format to reduce both memory and time complexity
  return images

def read_swimcat_data(data_folder):
  class_0 = load_images_from_folder(data_folder+'A-sky/images/')
  print('Class 0: ', len(class_0), class_0[0].shape)
  class_1 = load_images_from_folder(data_folder+'B-pattern/images/')
  print('Class 1: ', len(class_1), class_1[0].shape)
  class_2 = load_images_from_folder(data_folder+'C-thick-dark/images/')
  print('Class 2: ', len(class_2), class_2[0].shape)
  class_3 = load_images_from_folder(data_folder+'D-thick-white/images/')
  print('Class 3: ', len(class_3), class_3[0].shape)
  class_4 = load_images_from_folder(data_folder+'E-veil/images/')
  print('Class 4: ', len(class_4), class_4[0].shape)
  return class_0, class_1, class_2, class_3, class_4

class_0, class_1, class_2, class_3, class_4 = read_swimcat_data(data_folder)

Class 0:  224 (125, 125, 3)
Class 1:  89 (125, 125, 3)
Class 2:  251 (125, 125, 3)
Class 3:  135 (125, 125, 3)
Class 4:  85 (125, 125, 3)


Function to Augment Images

In [4]:
# Creates 144 tiled images from 1 original image - i.e. augment by a factor of 144
def augment_imageData(imageList, ori_row=125, ori_col=125, tiling=0.2):
  augmentedImages = []
  for image in imageList:
    if (not image.shape[0]==ori_row) or (not image.shape[1]==ori_col):
      print(image.shape)
      raise Exception("Dimensions of provided image doesn't match with specified dimensions!")
    # tiledImages = [leftTopCorner, centerTop, rightTopCorner,
    #                centerLeft, center, centerRight,
    #                leftBottomCorner, centerBottom, rightBottomCorner]
    tiledImages = [
                   image[0:int(ori_row*(1-tiling)), 0:int(ori_col*(1-tiling))],
                   image[0:int(ori_row*(1-tiling)), int(ori_col*(tiling/2)):int(ori_col*(1-(tiling/2)))],
                   image[0:int(ori_row*(1-tiling)), int(ori_col*tiling):ori_col],
                   image[int(ori_row*(tiling/2)):int(ori_row*(1-(tiling/2))), 0:int(ori_col*(1-tiling))],
                   image[int(ori_row*(tiling/2)):int(ori_row*(1-(tiling/2))), int(ori_col*(tiling/2)):int(ori_col*(1-(tiling/2)))],
                   image[int(ori_row*(tiling/2)):int(ori_row*(1-(tiling/2))), int(ori_col*tiling):ori_col],
                   image[int(ori_row*tiling):ori_row, 0:int(ori_col*(1-tiling))],
                   image[int(ori_row*tiling):ori_row, int(ori_col*(tiling/2)):int(ori_col*(1-(tiling/2)))],
                   image[int(ori_row*tiling):ori_row, int(ori_col*tiling):ori_col]
                  ]
    frtImages = []
    for tiledImage in tiledImages:
      if (not tiledImage.shape[0]==ori_row*(1-tiling)) or (not tiledImage.shape[1]==ori_col*(1-tiling)):
        raise Exception("Shape inconsistency error during tiling operation!")
      flippedImages = [
                       tiledImage, # Original
                       cv2.flip(src=tiledImage, flipCode=0).astype('uint8'), # Vertical Flip - around x-axis
                       cv2.flip(src=tiledImage, flipCode=1).astype('uint8'), # Horizontal Flip - around y-axis
                       cv2.flip(src=tiledImage, flipCode=-1).astype('uint8') # Both type of FLips
                      ]
      frImages = []
      for flippedImage in flippedImages:
        if (not flippedImage.shape[0]==ori_row*(1-tiling)) or (not flippedImage.shape[1]==ori_col*(1-tiling)):
          raise Exception("Shape inconsistency error during tiling operation!")
        rotatedImages = [
                         flippedImage, # Original
                         cv2.rotate(flippedImage, cv2.ROTATE_90_COUNTERCLOCKWISE).astype('uint8'), # 90 degree rotation
                         cv2.rotate(flippedImage, cv2.ROTATE_180).astype('uint8'), # 180 degree rotation
                         cv2.rotate(flippedImage, cv2.ROTATE_90_CLOCKWISE).astype('uint8') # 270 degree rotation
                        ]
        frImages.extend(rotatedImages)
      frtImages.extend(frImages)
    augmentedImages.extend(frtImages)
  return augmentedImages

# Function to extract center tile - for validation and testing purposes
def extract_center_tile(imageList, ori_row=125, ori_col=125, tiling=0.2):
  centerTiles = [None]*len(imageList)
  for id, image in enumerate(imageList):
    centerTiles[id] = image[int(ori_row*(tiling/2)):int(ori_row*(1-(tiling/2))), int(ori_col*(tiling/2)):int(ori_col*(1-(tiling/2)))]
    if (not centerTiles[id].shape[0]==ori_row*(1-tiling)) or (not centerTiles[id].shape[1]==ori_col*(1-tiling)):
        raise Exception("Shape inconsistency error during tiling operation!")
  return centerTiles

Create Train, Validation, and Test Splits

In [5]:
def create_splits(class_0, class_1, class_2, class_3, class_4, random_state=42, test_size=0.2):
  #---------------------------------------
  # Create Class-wise Initial Splits
  #---------------------------------------
  # Split Class 0
  X_train_complete_0, X_test_0, y_train_complete_0, y_test_0 = train_test_split(class_0, [np.array([1, 0, 0, 0, 0])]*len(class_0), random_state=random_state, test_size=test_size)
  X_train_0, X_val_0, y_train_0, y_val_0 = train_test_split(X_train_complete_0, y_train_complete_0, random_state=random_state, test_size=test_size)
  # Split Class 1
  X_train_complete_1, X_test_1, y_train_complete_1, y_test_1 = train_test_split(class_1, [np.array([0, 1, 0, 0, 0])]*len(class_1), random_state=random_state, test_size=test_size)
  X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X_train_complete_1, y_train_complete_1, random_state=random_state, test_size=test_size)
  # Split Class 2
  X_train_complete_2, X_test_2, y_train_complete_2, y_test_2 = train_test_split(class_2, [np.array([0, 0, 1, 0, 0])]*len(class_2), random_state=random_state, test_size=test_size)
  X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(X_train_complete_2, y_train_complete_2, random_state=random_state, test_size=test_size)
  # Split Class 3
  X_train_complete_3, X_test_3, y_train_complete_3, y_test_3 = train_test_split(class_3, [np.array([0, 0, 0, 1, 0])]*len(class_3), random_state=random_state, test_size=test_size)
  X_train_3, X_val_3, y_train_3, y_val_3 = train_test_split(X_train_complete_3, y_train_complete_3, random_state=random_state, test_size=test_size)
  # Split Class 4
  X_train_complete_4, X_test_4, y_train_complete_4, y_test_4 = train_test_split(class_4, [np.array([0, 0, 0, 0, 1])]*len(class_4), random_state=random_state, test_size=test_size)
  X_train_4, X_val_4, y_train_4, y_val_4 = train_test_split(X_train_complete_4, y_train_complete_4, random_state=random_state, test_size=test_size)
  #---------------------------------------
  # Augment Training Images
  #---------------------------------------
  # Augmenting Class 0
  X_train_0_aug = augment_imageData(X_train_0)
  y_train_0_aug = [np.array([1, 0, 0, 0, 0])]*len(X_train_0_aug)
  X_val_0_tiled = extract_center_tile(X_val_0)
  X_test_0_tiled = extract_center_tile(X_test_0)
  # Augmenting Class 1
  X_train_1_aug = augment_imageData(X_train_1)
  y_train_1_aug = [np.array([0, 1, 0, 0, 0])]*len(X_train_1_aug)
  X_val_1_tiled = extract_center_tile(X_val_1)
  X_test_1_tiled = extract_center_tile(X_test_1)
  # Augmenting Class 2
  X_train_2_aug = augment_imageData(X_train_2)
  y_train_2_aug = [np.array([0, 0, 1, 0, 0])]*len(X_train_2_aug)
  X_val_2_tiled = extract_center_tile(X_val_2)
  X_test_2_tiled = extract_center_tile(X_test_2)
  # Augmenting Class 3
  X_train_3_aug = augment_imageData(X_train_3)
  y_train_3_aug = [np.array([0, 0, 0, 1, 0])]*len(X_train_3_aug)
  X_val_3_tiled = extract_center_tile(X_val_3)
  X_test_3_tiled = extract_center_tile(X_test_3)
  # Augmenting Class 4
  X_train_4_aug = augment_imageData(X_train_4)
  y_train_4_aug = [np.array([0, 0, 0, 0, 1])]*len(X_train_4_aug)
  X_val_4_tiled = extract_center_tile(X_val_4)
  X_test_4_tiled = extract_center_tile(X_test_4)
  #---------------------------------------
  # Create Final Splits
  #---------------------------------------
  # Complete Train Split (NS - abbreviation for NOT SHUFFLED)
  X_train_NS = X_train_0_aug + X_train_1_aug + X_train_2_aug + X_train_3_aug + X_train_4_aug
  y_train_NS = y_train_0_aug + y_train_1_aug + y_train_2_aug + y_train_3_aug + y_train_4_aug
  temp = list(zip(X_train_NS, y_train_NS))
  random.shuffle(temp)
  X_train, y_train = zip(*temp)
  X_val_NS = X_val_0_tiled + X_val_1_tiled + X_val_2_tiled + X_val_3_tiled + X_val_4_tiled
  y_val_NS = y_val_0 + y_val_1 + y_val_2 + y_val_3 + y_val_4
  temp = list(zip(X_val_NS, y_val_NS))
  random.shuffle(temp)
  X_val, y_val = zip(*temp)
  X_test_NS = X_test_0_tiled + X_test_1_tiled + X_test_2_tiled + X_test_3_tiled + X_test_4_tiled
  y_test_NS = y_test_0 + y_test_1 + y_test_2 + y_test_3 + y_test_4
  temp = list(zip(X_test_NS, y_test_NS))
  random.shuffle(temp)
  X_test, y_test = zip(*temp)
  # Return splits
  return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = create_splits(class_0, class_1, class_2, class_3, class_4)
print('Number of images in Training Set = ', len(X_train))
print('Number of images in Validation Set = ', len(X_val))
print('Number of images in Test Set = ', len(X_test))

Number of images in Training Set =  71856
Number of images in Validation Set =  127
Number of images in Test Set =  158


## **Training Deep-CNN Model**

Declare Model Architecture and Hyper-Parameter Settings

In [6]:
results_folder = "/content/drive/My Drive/Colab Notebooks/Cloud Classification/results/"

input_shape = X_train[0].shape
num_classes = 5

NB_EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 1e-7

conv_base = VGG16(weights='imagenet',include_top=False,input_shape=input_shape)
model = Sequential()
model.add(conv_base)
model.add (Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(num_classes,activation="softmax"))
model.summary()
opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

58892288/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 15,895,877
Trainable params: 15,895,877
Non-trainable params: 0
_________________________________________________________________


Start Training

In [7]:
checkpoint = ModelCheckpoint(results_folder+"checkpointWeightsBest.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger(results_folder+"model_history_log.csv", append=False)

json_log = open(results_folder+'loss_log.json', mode='wt', buffering=1)
json_logging_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: json_log.write(
        json.dumps({'epoch': epoch, 'loss': logs['loss'], 'val_loss': logs['val_loss'], 'acc': logs['accuracy'], 'val_acc': logs['val_accuracy']}) + '\n'),
    on_train_end=lambda logs: json_log.close()
)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

callbacks_list = [checkpoint, csv_logger, json_logging_callback, es]

history = model.fit(np.asarray(X_train), np.asarray(y_train), validation_data=(np.asarray(X_val), np.asarray(y_val)), epochs=NB_EPOCHS, callbacks=callbacks_list)

Epoch 1/100
2246/2246 [==============================] - 234s 103ms/step - loss: 2.7431 - accuracy: 0.5187 - val_loss: 0.3577 - val_accuracy: 0.8898

Epoch 00001: val_accuracy improved from -inf to 0.88976, saving model to /content/drive/My Drive/Colab Notebooks/Cloud Classification/results/checkpointWeightsBest.hdf5
Epoch 2/100
2246/2246 [==============================] - 229s 102ms/step - loss: 0.2039 - accuracy: 0.9341 - val_loss: 0.1157 - val_accuracy: 0.9606

Epoch 00002: val_accuracy improved from 0.88976 to 0.96063, saving model to /content/drive/My Drive/Colab Notebooks/Cloud Classification/results/checkpointWeightsBest.hdf5
Epoch 3/100
2246/2246 [==============================] - 229s 102ms/step - loss: 0.0614 - accuracy: 0.9808 - val_loss: 0.0678 - val_accuracy: 0.9606

Epoch 00003: val_accuracy did not improve from 0.96063
Epoch 4/100
2246/2246 [==============================] - 230s 102ms/step - loss: 0.0244 - accuracy: 0.9932 - val_loss: 0.0564 - val_accuracy: 0.9685

Epoc

Resume Training

In [ ]:
checkpoint = ModelCheckpoint(results_folder+"checkpointWeightsBest.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger(results_folder+"model_history_log.csv", append=True)

json_log = open(results_folder+'loss_log.json', mode='wt', buffering=1)
json_logging_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: json_log.write(
        json.dumps({'epoch': epoch, 'loss': logs['loss'], 'val_loss': logs['val_loss'], 'acc': logs['accuracy'], 'val_acc': logs['val_accuracy']}) + '\n'),
    on_train_end=lambda logs: json_log.close()
)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

callbacks_list = [checkpoint, csv_logger, json_logging_callback, es]

model = load_model(results_folder+"checkpointWeightsBest.hdf5")

history = model.fit(np.asarray(X_train), np.asarray(y_train), validation_data=(np.asarray(X_val), np.asarray(y_val)), epochs=NB_EPOCHS, callbacks=callbacks_list)

Plot and Save Training Curves

In [7]:
training_logs = {'epoch'        : np.array([]),
                 'loss'         : np.array([]),
                 'val_loss'     : np.array([]),
                 'accuracy'     : np.array([]),
                 'val_accuracy' : np.array([])}
with open(results_folder+'loss_log.json') as logsJSON:
  lines = logsJSON.readlines()
  for line in lines:
    logsDict = json.loads(line)
    training_logs['epoch'] = np.append(training_logs['epoch'], logsDict['epoch'])
    training_logs['loss'] = np.append(training_logs['loss'], logsDict['loss'])
    training_logs['val_loss'] = np.append(training_logs['val_loss'], logsDict['val_loss'])
    training_logs['accuracy'] = np.append(training_logs['accuracy'], logsDict['acc'])
    training_logs['val_accuracy'] = np.append(training_logs['val_accuracy'], logsDict['val_acc'])

# Plot Loss Curves
plt.figure(figsize=(10, 6))
plt.rc('font', size=17)         # controls default text sizes
plt.rc('axes', titlesize=19)    # fontsize of the axes title
plt.rc('axes', labelsize=19)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=17)   # fontsize of the tick labels
plt.rc('ytick', labelsize=17)   # fontsize of the tick labels
plt.rc('legend', fontsize=17)   # legend fontsize
plt.rc('figure', titlesize=19)  # fontsize of the figure title
plt.plot(training_logs['epoch'], training_logs['loss'], 'r--', label='Training Loss')
plt.plot(training_logs['epoch'], training_logs['val_loss'], 'b--', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig(results_folder+'training_loss_characteristics.pdf', bbox_inches = 'tight', pad_inches = 0.05)
plt.close()

# Plot Accuracy Curves
plt.figure(figsize=(10, 6))
plt.rc('font', size=15)         # controls default text sizes
plt.rc('axes', titlesize=17)    # fontsize of the axes title
plt.rc('axes', labelsize=17)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=15)   # fontsize of the tick labels
plt.rc('ytick', labelsize=15)   # fontsize of the tick labels
plt.rc('legend', fontsize=15)   # legend fontsize
plt.rc('figure', titlesize=17)  # fontsize of the figure title
plt.plot(training_logs['epoch'], training_logs['accuracy'], 'r--', label='Training Accuracy')
plt.plot(training_logs['epoch'], training_logs['val_accuracy'], 'b--', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(results_folder+'training_accuracy_characteristics.pdf', bbox_inches = 'tight', pad_inches = 0.05)
plt.close()

Caluculate Loss and Accuracy on Test Set

In [8]:
model = load_model(results_folder+"checkpointWeightsBest.hdf5")
print("\nEvaluating on training data...")
results = model.evaluate(np.asarray(X_train), np.asarray(y_train), batch_size=BATCH_SIZE)
print("Train loss, Train accuracy:", results)
print("\nEvaluating on validation data...")
results = model.evaluate(np.asarray(X_val), np.asarray(y_val))
print("Val loss, Val accuracy:", results)
print("\nEvaluating on test data...")
results = model.evaluate(np.asarray(X_test), np.asarray(y_test))
print("Test loss, Test accuracy:", results)


Evaluating on training data...
1123/1123 [==============================] - 84s 68ms/step - loss: 7.9447e-07 - accuracy: 1.0000
Train loss, Train accuracy: [7.944732374198793e-07, 1.0]

Evaluating on validation data...
4/4 [==============================] - 1s 180ms/step - loss: 0.0515 - accuracy: 0.9921
Val loss, Val accuracy: [0.05146964639425278, 0.9921259880065918]

Evaluating on test data...
5/5 [==============================] - 1s 141ms/step - loss: 7.1612e-04 - accuracy: 1.0000
Test loss, Test accuracy: [0.0007161226239986718, 1.0]
